In [4]:
import graph_suite as plot
import parser
reload(parser)
import material_analytics
reload(material_analytics)
import numpy as np

### Doing this with KMeans Clustering

In [98]:
#add to GUI
elastic, plastic = material_analytics.kmeanssplit(parser.stress_strain('ref/1150.dat').get_experimental_data()[:,:2])
plot.plotmult2D(elastic,plastic, title = 'Stress vs Strain', xtitle = "Strain ($\epsilon$)", ytitle= "Stress ($\sigma$)")

### Doing this with Classic method

In [127]:
def yield_classic(data, cutoff = 0.1):
    
    startpt = cutoff # can't be zero or log will freak out
    
    data = material_analytics.log_prep(data,cutoff = cutoff)
    bestfit = material_analytics.log_approx(data)
    domain = np.linspace(startpt,max(data[:,0]),100001)

    vals = material_analytics.combine_data(domain,bestfit(domain))

    deriv1st = material_analytics.get_slopes(material_analytics.combine_data(domain,bestfit(domain)))

    find_nearest = lambda array,value: array[(np.abs(array-value)).argmin()]

    ave_slope = (vals[-1,1]-vals[0,1])/(vals[-1,0]-vals[0,0])
    index_ave = np.where(deriv1st == find_nearest(deriv1st, ave_slope))
    log_bend = vals[index_ave][0]
    young_mod = (log_bend[1]-vals[0,1])/(log_bend[0]-vals[0,0])

    linear_mod_2 = lambda x: young_mod*(x-0.2) + vals[0,1] # we will find where this intersects our exp data
    samples2 = material_analytics.samplepoints(linear_mod_2,[startpt,max(vals[:,0])],1001)

    diff = np.abs(samples2[:,1] - vals[:,1])

    """We can call this the intersection, its the closest point"""
    leastindex = diff.argmin()
    cross_est = vals[leastindex].reshape(1,-1)[0,1]
    yieldstress = find_nearest(data[:,1].reshape(-1,1),cross_est)[0]

    yieldpt = data[np.where(data[:,1] == yieldstress)]
    return yieldpt[0]

data = parser.stress_strain('ref/HSRS/326').get_experimental_data()
yieldpt = yield_classic(data)
index = np.where(data == yieldpt)[0][0]

elastic, plastic = data[:index], data[index:]
plot.plotmult2D(elastic,plastic, title = 'Stress vs Strain', xtitle = "Strain ($\epsilon$)", ytitle= "Stress ($\sigma$)")


## With custom method

In [7]:
data = parser.stress_strain('ref/900.dat').get_experimental_data()
yieldpt = material_analytics.yield_stress(data)[0]

index = np.where(data == yieldpt)[0][0]
elastic, plastic = data[:index+1], data[index:]

model = material_analytics.stress_model(data)

plot.plotmult2D(elastic,plastic, marker2 = 'r-', title = 'Stress vs Strain', xtitle = "Strain ($\epsilon$)", ytitle= "Stress ($\sigma$)")

In [10]:
data = parser.stress_strain('ref/1150.dat').get_experimental_data()

yldpt = material_analytics.yield_stress(data, cutoff=0.0000)
plot.plotmult2D(data, yldpt)

In [128]:
data = parser.stress_strain('ref/1150.dat').get_experimental_data()

find_nearest = lambda array,value: array[(np.abs(array-value)).argmin()]
#logft = material_analytics.log_approx(data)
yld = yield_classic(data)

TypeError: 'NoneType' object has no attribute '__getitem__'